## Import packages

In [1]:
import pandas as pd
import numpy as np

# Display Setting
from IPython.display import display
pd.options.display.max_colwidth=100
pd.options.display.float_format="{:.2f}".format
pd.set_option("display.max_columns", None)
import warnings
warnings.simplefilter('ignore')

# Exploratory data analysis
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pl
import chart_studio.plotly as py
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot


sns.set_theme(style='darkgrid')  # default style
import tensorflow as tf
np.set_printoptions(precision=3, suppress=True)  # improve float readability
from sklearn import datasets


# Data preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc,roc_curve,classification_report,confusion_matrix,mean_absolute_error,mean_squared_error,root_mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA
import string
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
%matplotlib inline

## Read data

Import raw data

In [2]:
animal_intakes_raw_data = pd.read_csv("Austin_Animal_Center_Intakes.csv")
animal_outcomes_raw_data = pd.read_csv("Austin_Animal_Center_Outcomes.csv")

In [3]:
print("In the intake dataset, we have {} records with {} variables".format(*animal_intakes_raw_data.shape))
print("In the outcome dataset, we have {} records with {} variables".format(*animal_outcomes_raw_data.shape))

In the intake dataset, we have 173812 records with 12 variables
In the outcome dataset, we have 173775 records with 12 variables


In [4]:
animal_intakes_raw_data.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A521520,Nina,10/01/2013 07:51:00 AM,October 2013,Norht Ec in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,Border Terrier/Border Collie,White/Tan
1,A664235,NaN,10/01/2013 08:33:00 AM,October 2013,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
2,A664236,NaN,10/01/2013 08:33:00 AM,October 2013,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
3,A664237,NaN,10/01/2013 08:33:00 AM,October 2013,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
4,A664233,Stevie,10/01/2013 08:53:00 AM,October 2013,7405 Springtime in Austin (TX),Stray,Injured,Dog,Intact Female,3 years,Pit Bull Mix,Blue/White


In [5]:
animal_intakes_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173812 entries, 0 to 173811
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         173812 non-null  object
 1   Name              123821 non-null  object
 2   DateTime          173812 non-null  object
 3   MonthYear         173812 non-null  object
 4   Found Location    173812 non-null  object
 5   Intake Type       173812 non-null  object
 6   Intake Condition  173812 non-null  object
 7   Animal Type       173812 non-null  object
 8   Sex upon Intake   173811 non-null  object
 9   Age upon Intake   173812 non-null  object
 10  Breed             173812 non-null  object
 11  Color             173812 non-null  object
dtypes: object(12)
memory usage: 15.9+ MB


In [6]:
animal_intakes_raw_data.describe()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
count,173812,123821,173812,173812,173812,173812,173812,173812,173811,173812,173812,173812
unique,156287,29774,119722,140,70183,6,20,5,5,55,3006,661
top,A721033,Luna,09/23/2016 12:00:00 PM,June 2015,Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Domestic Shorthair Mix,Black/White
freq,33,761,64,2189,31541,119160,147141,94608,58996,28294,33665,17976


In [7]:
animal_outcomes_raw_data.head()
animal_outcomes_raw_data.info()
animal_outcomes_raw_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173775 entries, 0 to 173774
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         173775 non-null  object
 1   Date of Birth     173775 non-null  object
 2   Name              123991 non-null  object
 3   DateTime          173775 non-null  object
 4   MonthYear         173775 non-null  object
 5   Outcome Type      173729 non-null  object
 6   Outcome Subtype   79660 non-null   object
 7   Animal Type       173775 non-null  object
 8   Sex upon Outcome  173774 non-null  object
 9   Age upon Outcome  173766 non-null  object
 10  Breed             173775 non-null  object
 11  Color             173775 non-null  object
dtypes: object(12)
memory usage: 15.9+ MB


,Animal ID,Date of Birth,Name,DateTime,MonthYear,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
count,173775,173775,123991,173775,173775,173729,79660,173775,173774,173766,173775,173775
unique,156237,8692,29756,146382,140,11,26,5,5,55,3010,663
top,A721033,2016-05-01,Luna,2016-04-18T00:00:00-05:00,06-2019,Adoption,Partner,Dog,Neutered Male,1 year,Domestic Shorthair Mix,Black/White
freq,33,121,760,39,2244,84598,40410,94505,60933,28782,34039,17999


Light cleaning

In [8]:
animal_intakes_data = animal_intakes_raw_data.copy()

# Check Intakes ID uniqueness
animal_intakes_data['Readoption'] = animal_intakes_data.groupby(['Animal ID']).cumcount()
animal_intakes_data['Animal ID'] = animal_intakes_data.apply(lambda x : f"{x['Animal ID']}_{x['Readoption']}" if x['Readoption'] > 0 else x['Animal ID'],axis=1)

# Fill name as N/A if no name
animal_intakes_data['Intake Name'] = animal_intakes_data['Name'].fillna('Unknown')
# Format intake date
animal_intakes_data['Intake Date'] = pd.to_datetime(animal_intakes_data['DateTime'],errors = "coerce").dt.date
# Drop duplicated columns
animal_intakes_data.drop(['Name','DateTime','MonthYear'],axis=1,inplace=True)

In [9]:
animal_intakes_data

,Animal ID,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Readoption,Intake Name,Intake Date
0,A521520,Norht Ec in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,Border Terrier/Border Collie,White/Tan,0,Nina,2013-10-01
1,A664235,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,0,Unknown,2013-10-01
2,A664236,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,0,Unknown,2013-10-01
3,A664237,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,0,Unknown,2013-10-01
4,A664233,7405 Springtime in Austin (TX),Stray,Injured,Dog,Intact Female,3 years,Pit Bull Mix,Blue/White,0,Stevie,2013-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...
173807,A929690,8038 Exchange Dr in Austin (TX),Stray,Injured,Dog,Intact Male,2 years,Belgian Malinois,Brown/Black,0,Unknown,2025-05-03
173808,A929717,Austin (TX),Public Assist,Normal,Dog,Intact Male,1 year,Shih Tzu Mix,White/Blue,0,Unknown,2025-05-04
173809,A929724,7105 Providence Ave Apt 3 in Austin (TX),Stray,Normal,Other,Unknown,1 year,Rabbit Sh,Tan/White,0,Unknown,2025-05-04
173810,A929725,1501 Red River St in Austin (TX),Public Assist,Normal,Dog,Intact Male,10 years,Boxer Mix,Tan/White,0,Oswold,2025-05-04


In [10]:
animal_outcomes_data = animal_outcomes_raw_data.copy()

# Check ID uniqueness
animal_outcomes_data['Readoption'] = animal_outcomes_data.groupby(['Animal ID']).cumcount()
animal_outcomes_data['Animal ID'] = animal_outcomes_data.apply(lambda x : f"{x['Animal ID']}_{x['Readoption']}" if x['Readoption'] > 0 else x['Animal ID'],axis=1)

# change DOB dtype
animal_outcomes_data['Date of Birth'] = pd.to_datetime(animal_outcomes_data['Date of Birth'],errors = "coerce").dt.date
# Fill name as N/A if no name
animal_outcomes_data['Outcome Name'] = animal_outcomes_data['Name'].fillna('Unknown')
# Format outcome date
animal_outcomes_data['Outcome Date'] = pd.to_datetime(animal_outcomes_data['DateTime'],errors = "coerce").dt.date
# Drop duplicated columns
animal_outcomes_data.drop(['Name','DateTime','MonthYear'],axis=1,inplace=True)

In [12]:
animal_outcomes_data = animal_outcomes_data[['Animal ID','Outcome Date', 'Outcome Name', 'Date of Birth', 'Outcome Type', 'Outcome Subtype','Age upon Outcome']]

In [13]:
animal_data = pd.merge(animal_intakes_data,animal_outcomes_data,on='Animal ID',how='inner',suffixes=('_intake','_outcome'))

In [14]:
animal_data

,Animal ID,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Readoption,Intake Name,Intake Date,Outcome Date,Outcome Name,Date of Birth,Outcome Type,Outcome Subtype,Age upon Outcome
0,A521520,Norht Ec in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,Border Terrier/Border Collie,White/Tan,0,Nina,2013-10-01,NaT,Nina,2006-09-07,Return to Owner,NaN,7 years
1,A664235,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,0,Unknown,2013-10-01,NaT,Unknown,2013-09-24,Transfer,Partner,1 week
2,A664236,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,0,Unknown,2013-10-01,NaT,Unknown,2013-09-24,Transfer,Partner,1 week
3,A664237,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White,0,Unknown,2013-10-01,NaT,Unknown,2013-09-24,Transfer,Partner,1 week
4,A664233,7405 Springtime in Austin (TX),Stray,Injured,Dog,Intact Female,3 years,Pit Bull Mix,Blue/White,0,Stevie,2013-10-01,NaT,Stevie,2010-09-30,Euthanasia,Suffering,3 years
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172845,A929600,25204 Fawn Drive in Travis (TX),Stray,Neonatal,Cat,Unknown,4 days,Domestic Shorthair,Brown Tabby,0,Unknown,2025-05-02,NaT,Unknown,2025-04-28,Transfer,Partner,4 days
172846,A929602,25204 Fawn Drive in Travis (TX),Stray,Normal,Cat,Unknown,1 year,Domestic Shorthair,Black,0,Unknown,2025-05-02,NaT,Unknown,2024-05-02,Transfer,Partner,1 year
172847,A848454_1,Austin (TX),Owner Surrender,Normal,Cat,Spayed Female,11 years,Domestic Shorthair Mix,Orange Tabby,1,Olli,2025-05-02,NaT,Olli,2014-04-20,Adoption,NaN,11 years
172848,A929631,Fm 1327 And Bradshaw in Austin (TX),Stray,Injured,Dog,Spayed Female,16 years,Chihuahua Shorthair/Dachshund,Tan,0,Fiona,2025-05-02,NaT,Fiona,2008-08-15,Euthanasia,At Vet,16 years


In [ ]:
#### 
# callable age in weeks year and day 

## EDA 

""" 
Set up a for loop to iterate over each feature
Print the name of the feature (excel header)
Plot how many are empty/null/na/0
Print data type
If categorical:
Print top values, 
print unique values [na, 'missing', 'did not provide']
If numerical:
Print max, mean, plot distributions [ages may be off, duration may not make sense]
Put together a rough data dictionary, provide to stakeholders for feedback (share the cleanup load!)
Put together a correlation matrix
Spearman
Pearson
"""

In [16]:
col_dict = []
def col_info(df):
    for col in df:
        entry = {
            "feature":col,
            "dtype":str(df[col].dtype),
            "N/A values": df[col].isnull().sum(),
            "N/A values %": (df[col].isnull().mean()* 100).round(),
            "unique values":df[col].nunique(),
        }
        
        col_dict.append(entry)
    return col_dict

In [17]:
intake_col_info = col_info(animal_intakes_raw_data)

In [18]:
intake_col_info

[{'feature': 'Animal ID',
  'dtype': 'object',
  'N/A values': 0,
  'N/A values %': 0.0,
  'unique values': 156287},
 {'feature': 'Name',
  'dtype': 'object',
  'N/A values': 49991,
  'N/A values %': 29.0,
  'unique values': 29774},
 {'feature': 'DateTime',
  'dtype': 'object',
  'N/A values': 0,
  'N/A values %': 0.0,
  'unique values': 119722},
 {'feature': 'MonthYear',
  'dtype': 'object',
  'N/A values': 0,
  'N/A values %': 0.0,
  'unique values': 140},
 {'feature': 'Found Location',
  'dtype': 'object',
  'N/A values': 0,
  'N/A values %': 0.0,
  'unique values': 70183},
 {'feature': 'Intake Type',
  'dtype': 'object',
  'N/A values': 0,
  'N/A values %': 0.0,
  'unique values': 6},
 {'feature': 'Intake Condition',
  'dtype': 'object',
  'N/A values': 0,
  'N/A values %': 0.0,
  'unique values': 20},
 {'feature': 'Animal Type',
  'dtype': 'object',
  'N/A values': 0,
  'N/A values %': 0.0,
  'unique values': 5},
 {'feature': 'Sex upon Intake',
  'dtype': 'object',
  'N/A values':

#### Feature Engineering

In [ ]:
# Drop duplicated columns : DateTime,MonthYear

### Missing value:

# only one Sex Type is missing 
display(animal_intakes_data[animal_intakes_data['Sex upon Intake'].isnull()])
display(animal_intakes_data[animal_intakes_data['Intake Name'] == 'Diego'][['Intake Name','Sex upon Intake']].value_counts())
# Check all other dog named 'Diego' and all of them are Male. So we full the unknown sex to Intact Male
animal_intakes_data.loc[2375,'Sex upon Intake'] = "Intact Male"

### outliers
animal_intakes_data['Age upon Intake'].value_counts()
# Feature selection :
# Drop OutcomeSubtype
# drop Animal Type ! = "Dog" or "Cat"
animal_intakes_data['Animal Type'].value_counts()
# do not include covid period